In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

import pickle # to save/load objects
import sys
sys.path.append("../helper/") # go to parent dir
from functions_preprocess import *
from functions_model import *

%load_ext autoreload
%autoreload 2

### Data Preprocessing
Here, I'm getting, cleaning and processing the data. I intentionally mixed the bands with different characteristics. I thought this may prevent overfitting and provide better word pool. Data has the almost entire discography of the bands, even demos for some.  

I'm also creating two dictionaries here, `vocab_to_int` and `int_to_vocab`. The model need numerical representation of the characters to be able to compute weights, biases, etc.

In [2]:
# prepare training data
artist_list = ['metallica', 'megadeth', 'slayer', 'judaspriest', 'ironmaiden', 'motorhead', 
               'blacksabbath', 'testament', 'overkill', 'anthrax', 'pantera', 'icedearth',
               'manowar', 'annihilator', 'exodus', 'dreamtheater']
text = ''
folder_name = ''
for i in artist_list:
    text = text + combine_songs(i)
    folder_name = folder_name + i[:2] # I know this's not the greatest folder name
    
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
chars = np.array([vocab_to_int[c] for c in text], dtype=np.int32)
print(folder_name, "ready with", len(chars), "chars")

memesljuirmoblteovanpaicmaanexdr ready with 3079342 chars


### Saving
I need to save the objects I created so far. Remember, no orders in Python dictionaries. This means this notebook creates different dictionaries in each session. Because I need the use my model later, I have to save these lookup dictionaries.

In [3]:
# Saving the objects
f = open('checkpoints/{}/vars.pckl'.format(folder_name), 'wb')
pickle.dump([text, vocab, vocab_to_int, int_to_vocab, chars, artist_list], f, protocol=2)
f.close()

I highly encourage you to check the function `split_data` in `functions_model.py`. The target is the next character in the sequence.  
`x = chars[: n_batches*slice_size]`  
`y = chars[1: n_batches*slice_size + 1]`

In [4]:
# Split training and validation sets
train_x, train_y, val_x, val_y = split_data(chars, 10, 200) # default fraction for trainning is 0.9

In [5]:
# hyperparameters
batch_size = 100
num_steps = 100
lstm_size = 512
num_layers = 2
learning_rate = 0.001

In [6]:
start_time = time.time() # always measure the time
epochs = 40
save_every_n = 500
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps)

model = build_rnn(len(vocab), 
                  batch_size=batch_size,
                  num_steps=num_steps,
                  learning_rate=learning_rate,
                  lstm_size=lstm_size,
                  num_layers=num_layers)

saver = tf.train.Saver(max_to_keep=100)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer = tf.summary.FileWriter('./logs/{}/train'.format(folder_name), sess.graph)
    test_writer = tf.summary.FileWriter('./logs/{}/test'.format(folder_name))
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/lyr20.ckpt')
    
    n_batches = int(train_x.shape[1]/num_steps)
    iterations = n_batches * epochs
    for e in range(epochs):
        
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for b, (x, y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
            iteration = e*n_batches + b
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 0.5,
                    model.initial_state: new_state}
            summary, batch_loss, new_state, _ = sess.run([model.merged, model.cost, 
                                                          model.final_state, model.optimizer], 
                                                          feed_dict=feed)
            loss += batch_loss
            end = time.time()
            print('Epoch {}/{} '.format(e+1, epochs),
                  'Iteration {}/{}'.format(iteration, iterations),
                  'Training loss: {:.4f}'.format(loss/b),
                  '{:.4f} sec/batch'.format((end-start)))
            
            train_writer.add_summary(summary, iteration)
        
            if (iteration%save_every_n == 0) or (iteration == iterations):
                # Check performance, notice dropout has been set to 1 
                # because this is validation, not training, so we need everything
                val_loss = []
                new_state = sess.run(model.initial_state)
                for x, y in get_batch([val_x, val_y], num_steps):
                    feed = {model.inputs: x,
                            model.targets: y,
                            model.keep_prob: 1.,
                            model.initial_state: new_state}
                    summary, batch_loss, new_state = sess.run([model.merged, model.cost, 
                                                               model.final_state], feed_dict=feed)
                    val_loss.append(batch_loss)
                    
                test_writer.add_summary(summary, iteration)

                print('Validation loss:', np.mean(val_loss),
                      'Saving checkpoint!')
                saver.save(sess, "checkpoints/{}/i{}_l{}_{:.3f}.ckpt".format(folder_name, iteration, lstm_size, np.mean(val_loss)))

print(epochs, "epoch done in", round(time.time() - start_time, 2)/60, "mins")

Epoch 1/40  Iteration 1/11040 Training loss: 4.4761 0.9269 sec/batch
Epoch 1/40  Iteration 2/11040 Training loss: 4.4324 0.4878 sec/batch
Epoch 1/40  Iteration 3/11040 Training loss: 4.2446 0.4667 sec/batch
Epoch 1/40  Iteration 4/11040 Training loss: 4.3949 0.4858 sec/batch
Epoch 1/40  Iteration 5/11040 Training loss: 4.3521 0.4824 sec/batch
Epoch 1/40  Iteration 6/11040 Training loss: 4.2877 0.4763 sec/batch
Epoch 1/40  Iteration 7/11040 Training loss: 4.2215 0.4735 sec/batch
Epoch 1/40  Iteration 8/11040 Training loss: 4.1513 0.4875 sec/batch
Epoch 1/40  Iteration 9/11040 Training loss: 4.0862 0.4765 sec/batch
Epoch 1/40  Iteration 10/11040 Training loss: 4.0323 0.4958 sec/batch
Epoch 1/40  Iteration 11/11040 Training loss: 3.9828 0.4804 sec/batch
Epoch 1/40  Iteration 12/11040 Training loss: 3.9386 0.4785 sec/batch
Epoch 1/40  Iteration 13/11040 Training loss: 3.9028 0.4814 sec/batch
Epoch 1/40  Iteration 14/11040 Training loss: 3.8712 0.4815 sec/batch
Epoch 1/40  Iteration 15/1104

Epoch 1/40  Iteration 118/11040 Training loss: 3.3084 0.4754 sec/batch
Epoch 1/40  Iteration 119/11040 Training loss: 3.3071 0.4824 sec/batch
Epoch 1/40  Iteration 120/11040 Training loss: 3.3060 0.4754 sec/batch
Epoch 1/40  Iteration 121/11040 Training loss: 3.3049 0.4766 sec/batch
Epoch 1/40  Iteration 122/11040 Training loss: 3.3037 0.4745 sec/batch
Epoch 1/40  Iteration 123/11040 Training loss: 3.3025 0.4793 sec/batch
Epoch 1/40  Iteration 124/11040 Training loss: 3.3013 0.4763 sec/batch
Epoch 1/40  Iteration 125/11040 Training loss: 3.3001 0.4744 sec/batch
Epoch 1/40  Iteration 126/11040 Training loss: 3.2987 0.4744 sec/batch
Epoch 1/40  Iteration 127/11040 Training loss: 3.2974 0.4774 sec/batch
Epoch 1/40  Iteration 128/11040 Training loss: 3.2960 0.4766 sec/batch
Epoch 1/40  Iteration 129/11040 Training loss: 3.2947 0.4744 sec/batch
Epoch 1/40  Iteration 130/11040 Training loss: 3.2933 0.4733 sec/batch
Epoch 1/40  Iteration 131/11040 Training loss: 3.2919 0.4774 sec/batch
Epoch 

Epoch 1/40  Iteration 234/11040 Training loss: 3.0591 0.4795 sec/batch
Epoch 1/40  Iteration 235/11040 Training loss: 3.0564 0.4794 sec/batch
Epoch 1/40  Iteration 236/11040 Training loss: 3.0537 0.4785 sec/batch
Epoch 1/40  Iteration 237/11040 Training loss: 3.0510 0.4784 sec/batch
Epoch 1/40  Iteration 238/11040 Training loss: 3.0484 0.4815 sec/batch
Epoch 1/40  Iteration 239/11040 Training loss: 3.0459 0.4784 sec/batch
Epoch 1/40  Iteration 240/11040 Training loss: 3.0433 0.4794 sec/batch
Epoch 1/40  Iteration 241/11040 Training loss: 3.0407 0.4764 sec/batch
Epoch 1/40  Iteration 242/11040 Training loss: 3.0381 0.4784 sec/batch
Epoch 1/40  Iteration 243/11040 Training loss: 3.0354 0.4804 sec/batch
Epoch 1/40  Iteration 244/11040 Training loss: 3.0327 0.4773 sec/batch
Epoch 1/40  Iteration 245/11040 Training loss: 3.0301 0.4783 sec/batch
Epoch 1/40  Iteration 246/11040 Training loss: 3.0276 0.4844 sec/batch
Epoch 1/40  Iteration 247/11040 Training loss: 3.0251 0.4775 sec/batch
Epoch 

Epoch 2/40  Iteration 350/11040 Training loss: 2.2782 0.4841 sec/batch
Epoch 2/40  Iteration 351/11040 Training loss: 2.2771 0.4788 sec/batch
Epoch 2/40  Iteration 352/11040 Training loss: 2.2761 0.4790 sec/batch
Epoch 2/40  Iteration 353/11040 Training loss: 2.2751 0.4816 sec/batch
Epoch 2/40  Iteration 354/11040 Training loss: 2.2743 0.4773 sec/batch
Epoch 2/40  Iteration 355/11040 Training loss: 2.2730 0.4656 sec/batch
Epoch 2/40  Iteration 356/11040 Training loss: 2.2717 0.4750 sec/batch
Epoch 2/40  Iteration 357/11040 Training loss: 2.2702 0.4809 sec/batch
Epoch 2/40  Iteration 358/11040 Training loss: 2.2687 0.4752 sec/batch
Epoch 2/40  Iteration 359/11040 Training loss: 2.2674 0.4832 sec/batch
Epoch 2/40  Iteration 360/11040 Training loss: 2.2658 0.4768 sec/batch
Epoch 2/40  Iteration 361/11040 Training loss: 2.2645 0.4655 sec/batch
Epoch 2/40  Iteration 362/11040 Training loss: 2.2634 0.4751 sec/batch
Epoch 2/40  Iteration 363/11040 Training loss: 2.2622 0.4859 sec/batch
Epoch 

Epoch 2/40  Iteration 466/11040 Training loss: 2.1903 0.4676 sec/batch
Epoch 2/40  Iteration 467/11040 Training loss: 2.1896 0.4772 sec/batch
Epoch 2/40  Iteration 468/11040 Training loss: 2.1888 0.4795 sec/batch
Epoch 2/40  Iteration 469/11040 Training loss: 2.1881 0.4904 sec/batch
Epoch 2/40  Iteration 470/11040 Training loss: 2.1875 0.4871 sec/batch
Epoch 2/40  Iteration 471/11040 Training loss: 2.1870 0.4788 sec/batch
Epoch 2/40  Iteration 472/11040 Training loss: 2.1864 0.4830 sec/batch
Epoch 2/40  Iteration 473/11040 Training loss: 2.1857 0.4915 sec/batch
Epoch 2/40  Iteration 474/11040 Training loss: 2.1851 0.4780 sec/batch
Epoch 2/40  Iteration 475/11040 Training loss: 2.1847 0.4727 sec/batch
Epoch 2/40  Iteration 476/11040 Training loss: 2.1841 0.4810 sec/batch
Epoch 2/40  Iteration 477/11040 Training loss: 2.1834 0.4764 sec/batch
Epoch 2/40  Iteration 478/11040 Training loss: 2.1827 0.4806 sec/batch
Epoch 2/40  Iteration 479/11040 Training loss: 2.1822 0.4684 sec/batch
Epoch 

...

Epoch 39/40  Iteration 10524/11040 Training loss: 1.1593 0.4770 sec/batch
Epoch 39/40  Iteration 10525/11040 Training loss: 1.1592 0.4723 sec/batch
Epoch 39/40  Iteration 10526/11040 Training loss: 1.1586 0.4726 sec/batch
Epoch 39/40  Iteration 10527/11040 Training loss: 1.1580 0.4608 sec/batch
Epoch 39/40  Iteration 10528/11040 Training loss: 1.1579 0.4733 sec/batch
Epoch 39/40  Iteration 10529/11040 Training loss: 1.1575 0.4756 sec/batch
Epoch 39/40  Iteration 10530/11040 Training loss: 1.1565 0.4757 sec/batch
Epoch 39/40  Iteration 10531/11040 Training loss: 1.1565 0.4734 sec/batch
Epoch 39/40  Iteration 10532/11040 Training loss: 1.1555 0.4741 sec/batch
Epoch 39/40  Iteration 10533/11040 Training loss: 1.1548 0.4775 sec/batch
Epoch 39/40  Iteration 10534/11040 Training loss: 1.1539 0.4737 sec/batch
Epoch 39/40  Iteration 10535/11040 Training loss: 1.1529 0.4754 sec/batch
Epoch 39/40  Iteration 10536/11040 Training loss: 1.1529 0.4746 sec/batch
Epoch 39/40  Iteration 10537/11040 Tra

Epoch 39/40  Iteration 10635/11040 Training loss: 1.1509 0.4756 sec/batch
Epoch 39/40  Iteration 10636/11040 Training loss: 1.1510 0.4757 sec/batch
Epoch 39/40  Iteration 10637/11040 Training loss: 1.1509 0.4891 sec/batch
Epoch 39/40  Iteration 10638/11040 Training loss: 1.1509 0.4771 sec/batch
Epoch 39/40  Iteration 10639/11040 Training loss: 1.1508 0.4743 sec/batch
Epoch 39/40  Iteration 10640/11040 Training loss: 1.1510 0.4752 sec/batch
Epoch 39/40  Iteration 10641/11040 Training loss: 1.1511 0.4746 sec/batch
Epoch 39/40  Iteration 10642/11040 Training loss: 1.1513 0.4757 sec/batch
Epoch 39/40  Iteration 10643/11040 Training loss: 1.1513 0.4746 sec/batch
Epoch 39/40  Iteration 10644/11040 Training loss: 1.1513 0.4617 sec/batch
Epoch 39/40  Iteration 10645/11040 Training loss: 1.1512 0.4735 sec/batch
Epoch 39/40  Iteration 10646/11040 Training loss: 1.1513 0.4753 sec/batch
Epoch 39/40  Iteration 10647/11040 Training loss: 1.1515 0.4762 sec/batch
Epoch 39/40  Iteration 10648/11040 Tra

Epoch 39/40  Iteration 10746/11040 Training loss: 1.1544 0.4744 sec/batch
Epoch 39/40  Iteration 10747/11040 Training loss: 1.1544 0.4765 sec/batch
Epoch 39/40  Iteration 10748/11040 Training loss: 1.1545 0.4742 sec/batch
Epoch 39/40  Iteration 10749/11040 Training loss: 1.1546 0.4604 sec/batch
Epoch 39/40  Iteration 10750/11040 Training loss: 1.1548 0.4731 sec/batch
Epoch 39/40  Iteration 10751/11040 Training loss: 1.1549 0.4745 sec/batch
Epoch 39/40  Iteration 10752/11040 Training loss: 1.1550 0.4604 sec/batch
Epoch 39/40  Iteration 10753/11040 Training loss: 1.1550 0.4749 sec/batch
Epoch 39/40  Iteration 10754/11040 Training loss: 1.1552 0.4752 sec/batch
Epoch 39/40  Iteration 10755/11040 Training loss: 1.1554 0.4751 sec/batch
Epoch 39/40  Iteration 10756/11040 Training loss: 1.1554 0.4588 sec/batch
Epoch 39/40  Iteration 10757/11040 Training loss: 1.1555 0.4762 sec/batch
Epoch 39/40  Iteration 10758/11040 Training loss: 1.1556 0.4733 sec/batch
Epoch 39/40  Iteration 10759/11040 Tra

Epoch 40/40  Iteration 10857/11040 Training loss: 1.1390 0.4742 sec/batch
Epoch 40/40  Iteration 10858/11040 Training loss: 1.1393 0.4770 sec/batch
Epoch 40/40  Iteration 10859/11040 Training loss: 1.1397 0.4760 sec/batch
Epoch 40/40  Iteration 10860/11040 Training loss: 1.1400 0.4723 sec/batch
Epoch 40/40  Iteration 10861/11040 Training loss: 1.1406 0.4755 sec/batch
Epoch 40/40  Iteration 10862/11040 Training loss: 1.1409 0.4763 sec/batch
Epoch 40/40  Iteration 10863/11040 Training loss: 1.1412 0.4901 sec/batch
Epoch 40/40  Iteration 10864/11040 Training loss: 1.1410 0.4774 sec/batch
Epoch 40/40  Iteration 10865/11040 Training loss: 1.1412 0.4722 sec/batch
Epoch 40/40  Iteration 10866/11040 Training loss: 1.1415 0.4763 sec/batch
Epoch 40/40  Iteration 10867/11040 Training loss: 1.1412 0.4613 sec/batch
Epoch 40/40  Iteration 10868/11040 Training loss: 1.1415 0.4725 sec/batch
Epoch 40/40  Iteration 10869/11040 Training loss: 1.1414 0.4903 sec/batch
Epoch 40/40  Iteration 10870/11040 Tra

Epoch 40/40  Iteration 10968/11040 Training loss: 1.1492 0.4595 sec/batch
Epoch 40/40  Iteration 10969/11040 Training loss: 1.1492 0.4755 sec/batch
Epoch 40/40  Iteration 10970/11040 Training loss: 1.1492 0.4719 sec/batch
Epoch 40/40  Iteration 10971/11040 Training loss: 1.1492 0.4770 sec/batch
Epoch 40/40  Iteration 10972/11040 Training loss: 1.1493 0.4749 sec/batch
Epoch 40/40  Iteration 10973/11040 Training loss: 1.1491 0.4746 sec/batch
Epoch 40/40  Iteration 10974/11040 Training loss: 1.1491 0.4639 sec/batch
Epoch 40/40  Iteration 10975/11040 Training loss: 1.1491 0.4691 sec/batch
Epoch 40/40  Iteration 10976/11040 Training loss: 1.1491 0.4762 sec/batch
Epoch 40/40  Iteration 10977/11040 Training loss: 1.1491 0.4752 sec/batch
Epoch 40/40  Iteration 10978/11040 Training loss: 1.1491 0.4740 sec/batch
Epoch 40/40  Iteration 10979/11040 Training loss: 1.1490 0.4743 sec/batch
Epoch 40/40  Iteration 10980/11040 Training loss: 1.1489 0.4753 sec/batch
Epoch 40/40  Iteration 10981/11040 Tra

Here is the loss graph,
<img src="assets/loss_graph.JPG" width="900">

### Test
No test here, because I'm not predicting house prices. The model is to generate, not to predict.

In [7]:
# list all checkpoints
tf.train.get_checkpoint_state('checkpoints/{}'.format(folder_name))

model_checkpoint_path: "checkpoints/memesljuirmoblteovanpaicmaanexdr\\i11040_l512_1.233.ckpt"
all_model_checkpoint_paths: "checkpoints/memesljuirmoblteovanpaicmaanexdr\\i500_l512_1.944.ckpt"
all_model_checkpoint_paths: "checkpoints/memesljuirmoblteovanpaicmaanexdr\\i1000_l512_1.637.ckpt"
all_model_checkpoint_paths: "checkpoints/memesljuirmoblteovanpaicmaanexdr\\i1500_l512_1.501.ckpt"
all_model_checkpoint_paths: "checkpoints/memesljuirmoblteovanpaicmaanexdr\\i2000_l512_1.428.ckpt"
all_model_checkpoint_paths: "checkpoints/memesljuirmoblteovanpaicmaanexdr\\i2500_l512_1.378.ckpt"
all_model_checkpoint_paths: "checkpoints/memesljuirmoblteovanpaicmaanexdr\\i3000_l512_1.353.ckpt"
all_model_checkpoint_paths: "checkpoints/memesljuirmoblteovanpaicmaanexdr\\i3500_l512_1.333.ckpt"
all_model_checkpoint_paths: "checkpoints/memesljuirmoblteovanpaicmaanexdr\\i4000_l512_1.313.ckpt"
all_model_checkpoint_paths: "checkpoints/memesljuirmoblteovanpaicmaanexdr\\i4500_l512_1.298.ckpt"
all_model_checkpoint_path

In [17]:
checkpoint = "checkpoints/{}/i10500_l512_1.231.ckpt".format(folder_name)
samp = sample(checkpoint, 700, lstm_size, len(vocab), vocab_to_int, vocab, int_to_vocab, prime="The ", top_n=3)
print(samp)

The Skin And The Stand
The Painkiller
The core was all the truth

There's no tomorrow, will never live to stop
The world is a conscience



Trapped us to the blood and the son and stare
The sun is calling to you
And the street of tomorrow's bleeding and darkness
Where the cores of the desires too much
I would be a shot or start to stone by the sky

I walk the earth to strength when the world's so falling
In the stars too long to the streets
And the fallen calls come for the same as they say the will to see

In my heart, I was a lie
The face that I see to save my sanity

I can't see my hands around me

I'm a most soul is all I want and I don't wanna see me anymore


There are the same and then the


In [27]:
checkpoint = "checkpoints/{}/i10020_l512_0.682.ckpt".format(folder_name)
samp = sample(checkpoint, 700, lstm_size, len(vocab), vocab_to_int, vocab, int_to_vocab, prime="Highway to Hell")
print(samp)

Highway to Hell
I am running ran away

In the night that search out
And the life I do is real
The sin I bring you where they soul
That he's suck a shame of someone else
I did to go down the fight
But they arconged straight one disgrace

I am the face that I'm going to
The fire in bed with someone else around

I walk always that you don't know it
If I can't take it any more again
The world is goena look behind all the storm
Why are you're feelings
In my sense that someone should haad
I'll never live the other day
It's my godna take that wounds
In the heart and that's all the way
This is the feot your sins of minutes
This is my war is all you'd die

And I'm going to be tile the proce is where you call
All thi


### Source
- [Udacity Deep Learning](https://github.com/udacity/deep-learning)  
- [Mat Leonard](https://github.com/mcleonard)